In [ ]:
%matplotlib inline

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
import matplotlib.pyplot as plt

from utils import (
    generate_hybrid_dataset,
    make_vqc_classifier,
    train_vqc,
    vqc_predict,
    make_kernel_circuit,
    compute_kernel_matrix,
    classical_rbf_svm,
    plot_decision_boundary,
)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
X, y, meta = generate_hybrid_dataset(n_samples=500, seed=RANDOM_SEED)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=RANDOM_SEED, stratify=y
)

X_train.shape, X_test.shape

In [ ]:
rbf_clf = classical_rbf_svm(C=1.0, gamma="scale")
rbf_clf.fit(X_train, y_train)

y_pred_rbf = rbf_clf.predict(X_test)
acc_rbf = accuracy_score(y_test, y_pred_rbf)
print(f"[Baseline] RBF SVM accuracy: {acc_rbf:.4f}")

In [ ]:
omega_qk = 10.0
kernel_fn = make_kernel_circuit(n_qubits=4, omega=omega_qk)

K_train = compute_kernel_matrix(kernel_fn, X_train, X_train)
K_test = compute_kernel_matrix(kernel_fn, X_test, X_train)

qk_clf = SVC(kernel="precomputed")
qk_clf.fit(K_train, y_train)

y_pred_qk = qk_clf.predict(K_test)
acc_qk = accuracy_score(y_test, y_pred_qk)
print(f"[Quantum Kernel] ω={omega_qk:.1f} accuracy: {acc_qk:.4f}")

In [ ]:
vqc_model = make_vqc_classifier(n_qubits=4, n_layers=2)

X_train_vqc = X_train
y_train_vqc = y_train.astype(float)

weights, vqc_losses = train_vqc(
    vqc_model,
    X_train_vqc,
    y_train_vqc,
    n_steps=80,
    lr=0.05,
    verbose=True,
)

print("Final VQC loss:", vqc_losses[-1])

In [ ]:
y_pred_vqc = vqc_predict(vqc_model, weights, X_test)
acc_vqc = accuracy_score(y_test, y_pred_vqc)

print(f"[VQC] accuracy: {acc_vqc:.4f}")
print(classification_report(y_test, y_pred_vqc))

In [ ]:
print("=== Comparative Accuracies ===")
print(f"Classical RBF SVM:     {acc_rbf:.4f}")
print(f"Quantum Kernel SVM:    {acc_qk:.4f} (ω={omega_qk:.1f})")
print(f"VQC (2-layer, 4-qubit): {acc_vqc:.4f}")

In [ ]:
plt.figure(figsize=(6,5))
plt.bar(["Classical RBF", "Kernel", "VQC"], [acc_rbf, acc_qk, acc_vqc], color=["gray","blue","purple"])
plt.ylabel("Accuracy")
plt.title("Comparative Accuracy Chart")
plt.tight_layout()
plt.savefig("../diagrams/comparison_barplot.png", dpi=300)
plt.show()